### 1) Content Ingestion 

### We need to deal with plethora of documents which is really necessary for the project. 

#### a) PDFs 
#### b) Word Documents
#### c) HTML pages 
#### d) Databases ( can be any) 

#### We can use document parsers such as Apache Tika ( personally I feel this is a good one), PyMuPDF, BeautifulSoup for web scraping. 
#### Another proposed solution is using LlamaIndex as they a variety of connectors available in LlamaHub
#### It is built in to LlamaIndex and can read a variety of formats including Markdown, PDFs, Word documents, PowerPoint decks, images, audio and video.
#### For our use case we can use loading using SimpleDirectoryReader

In [29]:
# System Utilities 
from spacy import displacy
import pymupdf
import requests
from bs4 import BeautifulSoup 
import re
import openai
import spacy
from spacy.lang.en import English

# LlamaIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core import Document
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display
from neo4j import GraphDatabase
from llama_index.core import StorageContext


from sentence_transformers import SentenceTransformer, util

import torch

In [23]:
!nvidia-smi

Thu Aug 15 19:43:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             31W /   70W |    1895MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
model_name = 'paraphrase-multilingual-mpnet-base-v2'

In [20]:
encoded_model_path = 'semantic_search_model.pt'

In [21]:
dataset_path = './SAP'

In [22]:
bi_encoder = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/envs/tensorflow/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
nlp = English()
nlp.add_pipe("sentencizer")

In [4]:
# :) The API Keys are removed after this project use. 
openai.api_key = "########################################################"

In [5]:
username = "neo4j"
password = "Me4JIgtHmDrCJLjTvSFxxzIyKPzCAOWxeUg4nZiaDxo"
url = "neo4j+s://cb52b641.databases.neo4j.io"

In [6]:
# Using PyMuPDF
doc = pymupdf.open("./CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf")
for page in doc:
    text = page.get_text()

In [7]:
text

'One of the key challenges for Celulose Nipo-Brasileira S.A. (CENIBRA) was increasing \nproductivity at a company that is already recognized globally for its high standard of \noperations. The company is a merge of two cultures, Brazilian and Japanese, that \nresulted in a unique vision of a consistent business and pragmatic quality improvements. \nSAP S/4HANA® was a natural choice for its transformation, replacing the old ERP system \nimplemented more than 17 years ago and integrating it with its logistics subsidiary, \nCENIBRA Logística. CENIBRA’s aim was to create both process synergies and new \nprocess designs to increase productivity.\nThe 13-month project covered all main processes, including sales, finance, production, \nHR, purchasing, warehouse, transportation, project document management, and treasury \nand risk management. Even with challenges due to the COVID-19 pandemic, the team \nwas able to work virtually and deliver the results as planned.\nCENIBRA simplified its proc

In [8]:
text[0] # Character level encoding

'O'

In [10]:
all_sentences = [] # Sentence Level Extraction

# Extract text from each page and apply the sentence splitter
for page in doc:
    text = page.get_text()  # Extract text from the page
    spacy_doc = nlp(text)   # Process the text with spaCy
    sentences = [sent.text for sent in spacy_doc.sents]  # Extract sentences
    all_sentences.extend(sentences)  # Add sentences to the list

# Print or process the sentences
count = 0
for sentence in all_sentences:
    print(sentence)
    count +=1
    
print(f'Count : {count}')

PUBLIC
Since 1975, Celulose Nipo-Brasileira S.A. (CENIBRA) has been processing pulp and 
producing paper products with a focus on sustainability, biodiversity, and traceability.
Pulp 
production requires large investments in capital, and asset utilization is a key component 
of a profitable producer.
In addition, industry volatility has increased due to economic 
turmoil that is impacting prices and logistical resources.

CENIBRA has a long history of achievements, however, it knew that transforming its IT 
landscape and processes was necessary to meet new market requirements and its 
business strategy.
One of the challenges CENIBRA faced was integration with the group’s 
logistics company, CENIBRA Logística.
The company also looked to create operational 
synergies, simplify processes, and increase productivity in areas including finance, 
maintenance, and operations.

CENIBRA: Taking the Next Steps 
Toward Sustainable Excellence with 
Intelligent Technologies
Picture Credit | Celulose

In [13]:
type(doc)

pymupdf.Document

In [14]:
# Iterate over the pages and extract text
text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)  # Load each page
    text += page.get_text("text")  # Extract text in simple text format

In [16]:
# Process the extracted text
doc = nlp(text)

# Iterate over tokens and print details
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

PUBLIC     XXXX True False

     
 False False
Since     Xxxxx True True
1975     dddd False False
,     , False False
Celulose     Xxxxx True False
Nipo     Xxxx True False
-     - False False
Brasileira     Xxxxx True False
S.A.     X.X. False False
(     ( False False
CENIBRA     XXXX True False
)     ) False False
has     xxx True True
been     xxxx True True
processing     xxxx True False
pulp     xxxx True False
and     xxx True True

     
 False False
producing     xxxx True False
paper     xxxx True False
products     xxxx True False
with     xxxx True True
a     x True True
focus     xxxx True False
on     xx True True
sustainability     xxxx True False
,     , False False
biodiversity     xxxx True False
,     , False False
and     xxx True True
traceability     xxxx True False
.     . False False
Pulp     Xxxx True False

     
 False False
production     xxxx True False
requires     xxxx True False
large     xxxx True False
investments     xxxx True False
in     xx True Tr

In [24]:
len(all_sentences)

24

### Creating our own Semantic Search Model

In [25]:
# Encode the sentences using SentenceTransformer
corpus_embeddings = bi_encoder.encode(
    all_sentences, batch_size=32, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# Save the encoded embeddings
torch.save(corpus_embeddings, encoded_model_path)

In [27]:
semantic_search_model = torch.load('semantic_search_model.pt')

/var/tmp/ipykernel_301154/3885871461.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  semantic_search_model = torch.load('semantic_search_model.pt')


In [33]:
question_embedding = bi_encoder.encode(
    "What is SAP 4HANA?", convert_to_tensor=True)
hits = util.semantic_search(
    question_embedding, semantic_search_model, top_k=3)
hits = hits[0]

result = {"search_results": [
    {"score": hit['score'], "text": all_sentences[hit['corpus_id']]} for hit in hits]}

In [34]:
result

{'search_results': [{'score': 0.6921055912971497,
   'text': 'This was one \nof the main drivers to migrate our SAP system to the \nlatest SAP S/4HANA software, a totally remodeled \nversion that’s faster and simpler and provides \nseveral possibilities to improve our processes.”'},
  {'score': 0.6767340302467346, 'text': '\nsap.com/terms-of-use\n'},
  {'score': 0.6698728203773499,
   'text': '\nSAP S/4HANA® was a natural choice for its transformation, replacing the old ERP system \nimplemented more than 17 years ago and integrating it with its logistics subsidiary, \nCENIBRA Logística.'}]}

### 2) Document parsing and Preprocessing

In [3]:
# Using LlamaIndex
documents = SimpleDirectoryReader("./SAP").load_data()

In [4]:
documents

[Document(id_='68564a1b-379c-4d6a-b127-aad93ea0ac12', embedding=None, metadata={'page_label': '1', 'file_name': 'CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_path': '/home/jupyter/Untitled Folder/SAP/CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_type': 'application/pdf', 'file_size': 434369, 'creation_date': '2024-08-15', 'last_modified_date': '2024-08-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A. (CENIBRA) has been processing pulp and \nproducing paper products with a focus on sustainability, biodiversity, and traceability. Pulp \nproduction requires large investments in capital, an

In [5]:
### Here there are 3 pages so the the length of this document is also 3. 
### This is divided based on the number of pages. 

#### Similarly we can also use Notion Connectors, Slack Connectors, Discord Connectors, Apify connector Which can crawl the web, scrape webpages, etract text content download files including .pdf, .jpg, .png , .docx unstructures data. 

### Now let's talk about meaningful extraction of data from the unstructured data. We can either chunk them up by sentences or of fixed length. Since chunking by sentences makes more coherent structures that would be more ideal. 

In [7]:
node_parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)

### Splitting by sentences 

### We extract out the set of nodes that will be stored in the VectorIndex. This includes both the nodes with the sentence window parser, as well as the "base" nodes extracted using the standard parser.

In [8]:
text_splitter = SentenceSplitter()

In [9]:
nodes = node_parser.get_nodes_from_documents(documents)

In [10]:
nodes

[TextNode(id_='b5d13ab3-68f0-49ec-bdd4-f04e2f2b3bd7', embedding=None, metadata={'page_label': '1', 'file_name': 'CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_path': '/home/jupyter/Untitled Folder/SAP/CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_type': 'application/pdf', 'file_size': 434369, 'creation_date': '2024-08-15', 'last_modified_date': '2024-08-15', 'window': 'PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A.  (CENIBRA) has been processing pulp and \nproducing paper products with a focus on sustainability, biodiversity, and traceability.  Pulp \nproduction requires large investments in capital, and asset utilization is a key component of a profitable producer.  In addition, industry volatility has increased due to economic \nturmoil that is impacting prices and logistical resources. \n', 'original_sentence': 'PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A. '}, 

In [11]:
len(nodes)

25

In [12]:
nodes[0]

TextNode(id_='b5d13ab3-68f0-49ec-bdd4-f04e2f2b3bd7', embedding=None, metadata={'page_label': '1', 'file_name': 'CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_path': '/home/jupyter/Untitled Folder/SAP/CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf', 'file_type': 'application/pdf', 'file_size': 434369, 'creation_date': '2024-08-15', 'last_modified_date': '2024-08-15', 'window': 'PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A.  (CENIBRA) has been processing pulp and \nproducing paper products with a focus on sustainability, biodiversity, and traceability.  Pulp \nproduction requires large investments in capital, and asset utilization is a key component of a profitable producer.  In addition, industry volatility has increased due to economic \nturmoil that is impacting prices and logistical resources. \n', 'original_sentence': 'PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A. '}, e

### Simple explanation here is that This text represents a TextNode object, which is part of the '''document processing''''.
### ID: 'a2468c97-f825-4b18-9bd0-e545a0960348'
### Embedding: None 
### Metadata: page_label: '1'
### file_name: 'CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf'
### file_path: '/home/jupyter/Untitled Folder/SAP/CENIBRA_ Taking the Next Steps Toward Sustainable Excellence with Intelligent Technologies.pdf'
### file_type: 'application/pdf'
### file_size: 434369
### creation_date: '2024-08-15'
### last_modified_date: '2024-08-15'
### window: Contains a larger text excerpt from the document
### original_sentence: 'PUBLIC\nSince 1975, Celulose Nipo -Brasileira S.A. '

### We will need to perform Manual Preprocessing before we pass text data to LlamaIndex, we can use libaries such as BeautifulSoup( for HTML parsing)
### and re(for regular expressions) to clean our text data. THis ensures that our data is free from unwanted characters and noise. Let's see an example. 

In [14]:
sentence_index = VectorStoreIndex(nodes)

In [15]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "List Value drivers in the Milling Industry for S/4HANA transformation, where also SAP EWM and TM was deployed"
)
print(window_response)

Single source of truth for customers and suppliers, flexible and faster cash-flow evaluation, improved operations through best practices from SAP Extended Warehouse Management and SAP Transportation Management applications, simplification of financial processes for accelerated monthly closing and greater financial insight, improvement in asset management for integrated management of depreciation and usage, and more efficient IT process management using SAP Solution Manager and focused developments.


In [17]:
def scrape_website(url):
    # Fetching the contant from website 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove the HTML tags and clean text Meanual Preprocessing
    text = soup.get_text()
    text = re.sub(r'\s+',' ',text)  # Replace multiple spaces with a single space
    return text 


In [18]:
def indexing_website_content(url):
    text = scrape_website(url)
    
    # Create a document for LlamaIndex
    document = Document(text=text)
    
    # Index Document 
    index = VectorStoreIndex([document])
    return index 

In [19]:
# Step 3: Querying the Index using Query Engine
def query_index(index, query):
    # Create a query engine from the index
    query_engine = index.as_query_engine()
    
    # Perform the query
    response = query_engine.query(query)
    return response

In [20]:
url = input("Enter the URL of the website you want to scrape: ")
index = indexing_website_content(url)

query = input("Enter your query: ")
response = query_index(index, query) 
print(f"Query Response:\n{response}")

Enter the URL of the website you want to scrape:  https://www.sap.com/index.html
Enter your query:  What does SAP stand for?


Query Response:
Systems, Applications, and Products in Data Processing.


## Solution 2 :Using Neo4j Graph Database

In [23]:
embed_dim = 1536
neo4j_vector = Neo4jVectorStore(username, password, url, embed_dim)

In [26]:
storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 80, offset: 79} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setVectorProperty(c, 'embedding', row.embedding) YIELD node SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS"


In [27]:
query_engine = index.as_query_engine()
response = query_engine.query("What is SAP 4HANA?")
display(Markdown(f"<b>{response}</b>"))

<b>SAP S/4HANA is a software solution that includes the latest digital capabilities to bring flexibility and productivity to an organization's IT department. It allows for a faster implementation, quality assurance, best practice selection, and knowledge transfer through the SAP Value Assurance program. Additionally, SAP S/4HANA and SAP Fiori apps help reduce manual efforts and streamline processes for improved efficiency.</b>

### Let's extract some entities and relationships that could potenttially be useful for constructing Knowledge graph 

In [31]:
nlp = spacy.load('en_core_web_sm')

In [32]:
# Process the text with spacy 
doc = nlp(text)

In [34]:
def extract_entities(doc):
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label))
    return entities 

In [46]:
# Function to extract relationships focusing on SVO structure
def extract_relationships(doc):
    relationships = []
    for token in doc:
        if token.dep_ == 'ROOT':  # Root verbs are often the main action in a sentence
            subject = None
            object_ = None
            
            # Look for the subject (nsubj)
            for child in token.lefts:
                if child.dep_ == 'nsubj':
                    subject = child.text
            
            # Look for the object (dobj, pobj)
            for child in token.rights:
                if child.dep_ in ('dobj', 'pobj'):
                    object_ = child.text
            
            if subject and object_:
                relationships.append((subject, token.lemma_, object_))
    return relationships


In [36]:
# Extract entities and relationships
entities = extract_entities(doc)
relationships = extract_relationships(doc)

In [37]:
# Display the extracted entities
print("Entities:")
for entity in entities:
    print(f" - {entity[0]} ({entity[1]})")

Entities:
 - One (397)
 - Celulose Nipo-Brasileira S.A. (383)
 - two (397)
 - Brazilian (381)
 - Japanese (381)
 - SAP (383)
 - ERP (383)
 - more than 17 years ago (391)
 - Logística (9191306739292312949)
 - CENIBRA (383)
 - 13-month (391)
 - treasury (383)
 - COVID-19 (383)
 - CENIBRA (383)
 - 320 (397)
 - Enabling Next-Level Productivity (383)
 - Faster (380)
 - 99% (393)
 - Simplification (383)
 - Ronaldo Neves Ribeiro (380)
 - Celulose Nipo-Brasileira 
S.A. (383)


### All these are not very useful entities

In [43]:
# Function to extract entities, filtered by type
def extract_entities(doc):
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON', 'GPE']:  # hERE WE ARE TRYING TO IDENTIFY RELEVANT ENTITIES
            entities.append((ent.text, ent.label_))
    return entities

In [44]:
entities = extract_entities(doc)

In [45]:
# Display the extracted entities
print("Filtered Entities:")
for entity in entities:
    print(f" - {entity[0]} ({entity[1]})")

Filtered Entities:
 - Celulose Nipo-Brasileira S.A. (ORG)
 - SAP (ORG)
 - ERP (ORG)
 - CENIBRA (ORG)
 - treasury (ORG)
 - COVID-19 (ORG)
 - CENIBRA (ORG)
 - Enabling Next-Level Productivity (ORG)
 - Faster (PERSON)
 - Simplification (ORG)
 - Ronaldo Neves Ribeiro (PERSON)
 - Celulose Nipo-Brasileira 
S.A. (ORG)


In [47]:
relationships = extract_relationships(doc)

In [48]:
# Display the extracted relationships
print("\nRelationships:")
for relationship in relationships:
    print(f" - {relationship[0]} {relationship[1]} {relationship[2]}")


Relationships:
 - One increase productivity
 - project cover processes
 - CENIBRA simplify processes


In [49]:
# Function to create nodes
def create_node(tx, entity, label):
    tx.run("MERGE (e:Entity {name: $name, label: $label})", name=entity, label=label)

In [50]:
# Function to create relationships
def create_relationship(tx, subject, verb, object_):
    tx.run("""
        MATCH (a:Entity {name: $subject}), (b:Entity {name: $object_})
        MERGE (a)-[r:RELATIONSHIP {type: $verb}]->(b)
    """, subject=subject, verb=verb, object_=object_)

In [51]:
# List of entities and their types
entities = [
    ("CENIBRA", "ORG"),
    ("Celulose Nipo-Brasileira S.A.", "ORG"),
    ("Ronaldo Neves Ribeiro", "PERSON"),
    ("SAP", "ORG"),
    ("COVID-19", "EVENT")
]

In [52]:
# List of relationships (subject, verb, object)
relationships = [
    ("CENIBRA", "simplify", "processes"),
    ("project", "cover", "all processes"),
    ("team", "work on", "transformation")
]

In [56]:
driver = GraphDatabase.driver(url, auth=(username, password))

In [58]:
# Create nodes and relationships in the graph
with driver.session() as session:
    for entity, label in entities:
        session.write_transaction(create_node, entity, label)
    
    for subject, verb, object_ in relationships:
        session.write_transaction(create_relationship, subject, verb, object_)

print("Graph construction complete!")

/var/tmp/ipykernel_212759/1211116460.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, entity, label)
/var/tmp/ipykernel_212759/1211116460.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, subject, verb, object_)


Graph construction complete!
